# Street Light Data

In [2]:
# import libraries
import pandas as pd
import numpy as np
import math
from collections import defaultdict

### Dictionary: the number of trips origined from each county (e.g. 17001)

for both annully aggregated data and street light data

In [9]:
# read the annully aggregated data
od_trips_df = pd.read_csv("risk_flow_data/origin_dest_trips_census_tract_level.csv")
od_trips_df

,RESIDENCE,WORKPLACE,JOBS
0,17001000100,17001000100,139
1,17001000100,17001000201,12
2,17001000100,17001000202,9
3,17001000100,17001000400,10
4,17001000100,17001000500,119
...,...,...,...
80423,17203030700,17113005102,2
80424,17203030700,17143001600,5
80425,17203030700,17179021201,2
80426,17203030700,17203030601,6


In [1]:
od_dict = defaultdict(int)
num_ct = od_trips_df.shape[0]
for i in range(num_ct):
    fip = int(od_trips_df.iloc[i,0]/10e5-17000)
    od_dict[fip]+= od_trips_df.iloc[i,2]     

NameError: name 'defaultdict' is not defined

In [3]:
# read the street light data
streetlight_trips_p3 = pd.read_csv("Streetlight_Data/streetlight_trips_p3.csv")[['Origin.ID', 'Destination.ID', 'Traffic.V']]
streetlight_trips_p3

,Origin.ID,Destination.ID,Traffic.V
0,31,31,14936569
1,43,43,2462848
2,197,197,1952331
3,97,97,1852828
4,201,201,1091282
...,...,...,...
7148,93,187,0
7149,95,45,0
7150,97,193,0
7151,97,35,0


In [11]:
# read the street light data
streetlight_trips_p4 = pd.read_csv("/Users/zhangyuqi/Documents/NCSA/Exposure_Risk_Analysis_in Rural_Illinois/Streetlight_Data/streetlight_trips_p4.csv")[['Origin.ID', 'Destination.ID', 'Traffic.V']]
streetlight_trips_p4

,Origin.ID,Destination.ID,Traffic.V
0,31,31,13744545
1,43,43,2403247
2,197,197,1813707
3,97,97,1767243
4,89,89,1026100
...,...,...,...
8755,99,185,0
8756,99,191,0
8757,99,3,0
8758,99,33,0


In [25]:
sl4_dict = defaultdict(float)
num_ct = streetlight_trips_p4.shape[0]
for i in range(num_ct):
    fip = int(streetlight_trips_p4.iloc[i,0])
    sl4_dict[fip]+= streetlight_trips_p4.iloc[i,2]     

### Double Dictionary: (county level) number of trips from each origin to each dest

In [26]:
#function to fo defaultdict(defaultdict(int))
# when {} is empty, enable
# df['a']['b']+=1
def defaultdict_int():
    return collections.defaultdict(int)

In [27]:
# anuallly aggregated data 
# origin -> destination (county level) frequency dictionary
od_od_dict = defaultdict(defaultdict_int)
num_ct = od_trips_df.shape[0]
for i in range(num_ct):
    fip_o = int(od_trips_df.iloc[i,0]/10e5-17000)
    fip_d = int(od_trips_df.iloc[i,1]/10e5-17000)
    od_od_dict[fip_o][fip_d] += od_trips_df.iloc[i,2]

In [28]:
# street light data 
# origin -> destination (county level) frequency dictionary
sl4_od_dict = defaultdict(defaultdict_int)
num_ct = streetlight_trips_p4.shape[0]
for i in range(num_ct):
    fip_o = int(streetlight_trips_p4.iloc[i,0])
    fip_d = int(streetlight_trips_p4.iloc[i,1])
    sl4_od_dict[fip_o][fip_d] += streetlight_trips_p4.iloc[i,2]

### Update the number of trips in anually aggregated data according to the number of trips in street light data

In [30]:
slod4_trips_df = od_trips_df.copy(deep=True)

num_ct = od_trips_df.shape[0]
for i in range(num_ct):
    fip_o = int(od_trips_df.iloc[i,0]/10e5-17000)
    fip_d = int(od_trips_df.iloc[i,1]/10e5-17000)
    if od_od_dict[fip_o][fip_d] == 0:# if the denorminator is 0
        multiplier = 0
    else:
        multiplier = sl4_od_dict[fip_o][fip_d]/od_od_dict[fip_o][fip_d]
    slod4_trips_df.iloc[i,2] = multiplier*od_trips_df.iloc[i,2]

In [31]:
slod4_trips_df 

,RESIDENCE,WORKPLACE,JOBS
0,17001000100,17001000100,4955.433246
1,17001000100,17001000201,427.807187
2,17001000100,17001000202,320.855390
3,17001000100,17001000400,356.505989
4,17001000100,17001000500,4242.421268
...,...,...,...
80423,17203030700,17113005102,47.255663
80424,17203030700,17143001600,35.376124
80425,17203030700,17179021201,152.861111
80426,17203030700,17203030601,469.589286


In [32]:
slod4_trips_df.to_csv("Streetlight_Data/clean_data/streetlight_OD_p4.csv")

In [22]:
slod4_dict = defaultdict(int)
num_ct = slod4_trips_df.shape[0]
for i in range(num_ct):
    fip = int(slod4_trips_df.iloc[i,0]/10e5-17000)
    slod4_dict[fip]+= slod4_trips_df.iloc[i,2]  

### Evaluation and EDA

#### similarity between street light data and annual aggregated data

In [17]:
num_ct = od_trips_df.shape[0]
nu = 0
de = 0
for i in range(num_ct):
    fip_o = int(od_trips_df.iloc[i,0]/10e5-17000)
    fip_d = int(od_trips_df.iloc[i,1]/10e5-17000)
    de += 1
    if od_od_dict[fip_o][fip_d] != 0 and sl4_od_dict[fip_o][fip_d] != 0:
        nu += 1
nu/de

0.9998632317103496

#### Calculate the RMSE and the absolute error

In [18]:
SSE = 0
for fip in slod4_dict:
    error = (slod4_dict[fip]-sl4_dict[fip])**2
    SSE += error
RMSE = math.sqrt(SSE/len(slod4_dict))
RMSE

2908.6405053120166

In [19]:
sl4_mean = np.mean(list(sl4_dict.values()))

In [20]:
RMSE/sl4_mean

0.007418071076689271

### Repeat the process to het the streetlight trips for period 1, 2, 3

In [40]:
def toStreetlightTrip(in_path, out_path):
    # read the street light data
    streetlight_trips_p = pd.read_csv(in_path)[['Origin.ID', 'Destination.ID', 'Traffic.V']]
    # street light data 
    # origin -> destination (county level) frequency dictionary
    sl_od_dict = defaultdict(defaultdict_int)
    num_ct = streetlight_trips_p.shape[0]
    for i in range(num_ct):
        fip_o = int(streetlight_trips_p.iloc[i,0])
        fip_d = int(streetlight_trips_p.iloc[i,1])
        sl_od_dict[fip_o][fip_d] += streetlight_trips_p.iloc[i,2]

    # Update the number of trips in anually aggregated data according to the number of trips in street light data
    slod_trips_df = od_trips_df.copy(deep=True)
    num_ct = od_trips_df.shape[0]
    for i in range(num_ct):
        fip_o = int(od_trips_df.iloc[i,0]/10e5-17000)
        fip_d = int(od_trips_df.iloc[i,1]/10e5-17000)
        if od_od_dict[fip_o][fip_d] == 0:# if the denorminator is 0
            multiplier = 0
        else:
            multiplier = sl_od_dict[fip_o][fip_d]/od_od_dict[fip_o][fip_d]
        slod_trips_df.iloc[i,2] = multiplier*od_trips_df.iloc[i,2]

        # save the preprocesed data into csv file
    slod_trips_df.to_csv(out_path)

In [41]:
toStreetlightTrip(in_path = "Streetlight_Data/streetlight_trips_p3.csv",
                  out_path = "Streetlight_Data/clean_data/streetlight_OD_p3.csv")

In [42]:
toStreetlightTrip(in_path = "Streetlight_Data/streetlight_trips_p2.csv",
                  out_path = "Streetlight_Data/clean_data/streetlight_OD_p2.csv")

In [43]:
toStreetlightTrip(in_path = "Streetlight_Data/streetlight_trips_p1.csv",
                  out_path = "Streetlight_Data/clean_data/streetlight_OD_p1.csv")